In [1]:
cd /media/data/Dropbox/Prog/python/Import/wyrm/

/media/data/Dropbox/Prog/python/Import/wyrm


In [2]:
ls

build/            test_plot_channels_cnt.jpg
doc/              test_plot_channels_epo.jpg
examples/         test_plot_scalp.jpg
foo.marker        test_plot_scalp_ti.jpg
LICENSE           test_plot_spatio_temporal_r2_values.jpg
Makefile          test_plot_spectrogram.jpg
README.md         test_plot_tenten.jpg
requirements.txt  test_plot_timeinterval.jpg
setup.py          THANKS
test/             TODO
TestData/         wyrm/


In [3]:
ls examples/

'BCI Competition 3, Data Set 1 (motor imagery in ECoG recordings).ipynb'
'BCI Competition 3, Data Set 2 (P300 Speller).ipynb'
 desc_II_data.pdf
'ERP Analysis.ipynb'
'ERP Classification.ipynb'
'MI Analysis.ipynb'
 online_experiment.ipynb
 online_experiment.py*
 output_online_experiment.txt
 performance.py*
 Untitled.ipynb


In [4]:
#!/usr/bin/env python


from __future__ import division

import sys
import time
import logging

import numpy as np


In [5]:
# change this to the path of mushu if you don't have it in your
# PYTHONPATH already
sys.path.append('../mushu')
# sys.path.append('')

import libmushu
from wyrm.types import RingBuffer
import wyrm.processing as proc
from wyrm import io

In [7]:




logging.basicConfig(format='%(relativeCreated)10.0f %(threadName)-10s %(name)-10s %(levelname)8s %(message)s', level=logging.NOTSET)
logger = logging.getLogger(__name__)

# replay the experiment in real time?
REALTIME = False


TRAIN_DATA = 'TestData/II/BCI_Comp_III_Wads_2004/Subject_A_Train.mat'
TEST_DATA = 'TestData/II/BCI_Comp_III_Wads_2004/Subject_A_Test.mat'

CHANNEL_DATA = 'TestData/II/BCI_Comp_III_Wads_2004/eloc64.txt'

TRUE_LABELS = "WQXPLZCOMRKO97YFZDEZ1DPI9NNVGRQDJCUVRMEUOOOJD2UFYPOO6J7LDGYEGOA5VHNEHBTXOO1TDOILUEE5BFAEEXAW_K4R3MRU"

STIMULUS_CODE = {
    # cols from left to right
    1 : "agmsy5",
    2 : "bhntz6",
    3 : "ciou17",
    4 : "djpv28",
    5 : "ekqw39",
    6 : "flrx4_",
    # rows from top to bottom
    7 : "abcdef",
    8 : "ghijkl",
    9 : "mnopqr",
    10: "stuvwx",
    11: "yz1234",
    12: "56789_"
}

MARKER_DEF_TRAIN = {'target': ['target'], 'nontarget': ['nontarget']}
MARKER_DEF_TEST = {i : [i] for i in STIMULUS_CODE.values()}

JUMPING_MEANS_IVALS = [150, 220], [200, 260], [310, 360], [550, 660] # 91%

SEG_IVAL = [0, 700]



In [19]:

def online_experiment(amp, cfy):
    amp_fs = amp.get_sampling_frequency()
    amp_channels = amp.get_channels()

    #buf = BlockBuffer(4)
    rb = RingBuffer(5000)

    fn = amp_fs / 2
    b_low, a_low = proc.signal.butter(5, [30 / fn], btype='low')
    b_high, a_high = proc.signal.butter(5, [.4 / fn], btype='high')

    zi_low = proc.lfilter_zi(b_low, a_low, len(amp_channels))
    zi_high = proc.lfilter_zi(b_high, a_high, len(amp_channels))

    amp.start()
    markers_processed = 0
    current_letter_idx = 0
    current_letter = TRUE_LABELS[current_letter_idx].lower()

    letter_prob = {i : 0 for i in 'abcdefghijklmnopqrstuvwxyz123456789_'}
    endresult = []
    t0 = time.time()
    while True:
        t0 = time.time()

        # get fresh data from the amp
        data, markers = amp.get_data()
        if len(data) == 0:
            continue

        # we should rather wait for a specific end-of-experiment marker
        if len(data) == 0:
            break

        # convert to cnt
        cnt = io.convert_mushu_data(data, markers, amp_fs, amp_channels)

        ## enter the block buffer
        #buf.append(cnt)
        #cnt = buf.get()
        #if not cnt:
        #    continue

        # band-pass and subsample
        cnt, zi_low = proc.lfilter(cnt, b_low, a_low, zi=zi_low)
        cnt, zi_high = proc.lfilter(cnt, b_high, a_high, zi=zi_high)

        cnt = proc.subsample(cnt, 60)

        newsamples = cnt.data.shape[0]
        # enter the ringbuffer
        rb.append(cnt)
        cnt = rb.get()

        # segment
        epo = proc.segment_dat(cnt, MARKER_DEF_TEST, SEG_IVAL, newsamples=newsamples)
        if not epo:
            continue

        fv = proc.jumping_means(epo, JUMPING_MEANS_IVALS)
        fv = proc.create_feature_vectors(fv)
        logger.debug(markers_processed)

        lda_out = proc.lda_apply(fv, cfy)
        markers = [fv.class_names[cls_idx] for cls_idx in fv.axes[0]]
        result = zip(markers, lda_out)
        for s, score in result:
            if markers_processed == 180:
                endresult.append(sorted(letter_prob.items(), key=lambda x: x[1])[-1][0])
                letter_prob = {i : 0 for i in 'abcdefghijklmnopqrstuvwxyz123456789_'}
                markers_processed = 0
                current_letter_idx += 1
                current_letter = TRUE_LABELS[current_letter_idx].lower()
            for letter in s:
                letter_prob[letter] += score
            markers_processed += 1
        logger.debug("".join([i[0] for i in sorted(letter_prob.items(), key=lambda x: x[1], reverse=True)]).replace(current_letter, " %s " % current_letter))
        logger.debug(TRUE_LABELS)
        logger.debug("".join(endresult))
        # calculate the current accuracy
        if len(endresult) > 0:
            acc = np.count_nonzero(np.array(endresult) == np.array(list(TRUE_LABELS.lower()[:len(endresult)]))) / len(endresult)
            print "Current accuracy:", acc * 100
        if len(endresult) == len(TRUE_LABELS):
            break
        #logger.debug("Result: %s" % result)
        print 1000 * (time.time() - t0)

    acc = np.count_nonzero(np.array(endresult) == np.array(list(TRUE_LABELS.lower()[:len(endresult)]))) / len(endresult)
    print "Accuracy:", acc * 100

    amp.stop()



In [20]:

def train(filename):
    cnt = io.load_bcicomp3_ds2(filename)

    fs_n = cnt.fs / 2

    b, a = proc.signal.butter(5, [30 / fs_n], btype='low')
    cnt = proc.lfilter(cnt, b, a)

    b, a = proc.signal.butter(5, [.4 / fs_n], btype='high')
    cnt = proc.lfilter(cnt, b, a)

    cnt = proc.subsample(cnt, 60)

    epo = proc.segment_dat(cnt, MARKER_DEF_TRAIN, SEG_IVAL)

    #from wyrm import plot
    #plot.plot_spatio_temporal_r2_values(proc.sort_channels(epo))
    #print JUMPING_MEANS_IVALS
    #plot.plt.show()

    fv = proc.jumping_means(epo, JUMPING_MEANS_IVALS)
    fv = proc.create_feature_vectors(fv)

    cfy = proc.lda_train(fv)
    return cfy


NameError: name 'TRAIN_DATA' is not defined

In [21]:
logger.debug('Training...')
cfy = train(TRAIN_DATA)

DEBUG:__main__:Training...


In [23]:
cfy

(array([  1.54514596e-02,   1.76694111e-02,   4.90852946e-03,
         -1.45513628e-02,  -7.62275416e-03,   3.83545302e-02,
         -2.90049379e-02,   2.56804992e-03,  -1.04432167e-02,
          9.69985681e-03,  -6.67797674e-03,   1.08654463e-02,
         -2.94367463e-03,  -1.03315660e-02,   1.62394143e-04,
          7.45730445e-03,  -1.01887868e-02,  -9.17934689e-03,
         -2.15850734e-02,   2.16196137e-02,   1.03426632e-02,
          3.79388207e-03,  -7.56718843e-04,   1.39559145e-02,
          2.68380016e-03,  -2.35456349e-02,  -1.43948935e-02,
          1.28863949e-02,  -2.30317474e-02,  -1.05189970e-02,
         -3.96679695e-03,   5.19968833e-03,   4.66323454e-02,
         -4.41702787e-02,   2.34538334e-02,   2.56753676e-02,
         -3.92052667e-04,   4.28023918e-03,  -1.52834425e-02,
         -4.03186060e-03,  -1.04150239e-02,  -1.13589401e-02,
          2.05316321e-04,  -1.89492084e-02,  -8.17597698e-03,
          1.80687926e-02,  -2.96585129e-03,   6.53228069e-03,
        

In [24]:
logger.debug('Starting Online experiment...')

DEBUG:__main__:Starting Online experiment...


In [26]:
cnt = io.load_bcicomp3_ds2(TEST_DATA)

In [27]:
amp = libmushu.get_amp('replayamp')

In [15]:
av_amp = libmushu.get_available_amps()

In [16]:
av_amp

['randomamp', 'sinusamp', 'replayamp']

In [30]:
amp.configure(data=cnt.data, marker=cnt.markers, channels=cnt.axes[-1], fs=cnt.fs, realtime=False, blocksize_samples=40)

In [17]:
amp = libmushu.get_amp('replayamp')

In [19]:
amp?